# Video2X Modern Setup - Docker Version

This notebook demonstrates how to use the modern Video2X (version 6+) which is now a C/C++ application running in Docker.

## Features
- **Real-ESRGAN**: Best for real-world videos
- **Anime4K**: Optimized for anime content
- **RIFE**: Frame interpolation for smoother video
- **Fast Processing**: C/C++ implementation is much faster than the old Python version

In [ ]:
# Import required libraries
import os
import sys
import subprocess
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Add our custom Video2X wrapper
sys.path.append('/workspaces/video2x-codespace')
import video2x_wrapper

In [ ]:
# Initialize Video2X wrapper
v2x = video2x_wrapper.Video2X()

print("📁 Workspace structure:")
print(f"Input directory: {v2x.input_dir}")
print(f"Output directory: {v2x.output_dir}")

# Create directories if they don't exist
v2x.input_dir.mkdir(exist_ok=True)
v2x.output_dir.mkdir(exist_ok=True)

print("\n✅ Video2X wrapper initialized!")

## Check Available Videos

In [ ]:
# List available input videos
input_files = list(v2x.input_dir.glob('*.mp4')) + list(v2x.input_dir.glob('*.avi')) + list(v2x.input_dir.glob('*.mkv'))

print("📹 Available videos in input directory:")
if input_files:
    for i, file in enumerate(input_files):
        print(f"  {i+1}. {file.name}")
else:
    print("  No videos found. Upload a video to the input/ directory first.")
    
    # Create a sample video if none exists
    print("\n🎥 Creating a sample test video...")
    try:
        subprocess.run([
            'ffmpeg', '-f', 'lavfi', '-i', 'testsrc=duration=5:size=320x240:rate=30',
            str(v2x.input_dir / 'sample_test.mp4'),
            '-y', '-loglevel', 'quiet'
        ], check=True)
        print("✅ Sample video created: sample_test.mp4")
        input_files = [v2x.input_dir / 'sample_test.mp4']
    except:
        print("❌ Could not create sample video")

## Video Analysis

In [ ]:
# Analyze a video file
if input_files:
    video_file = input_files[0]  # Use first available video
    print(f"📊 Analyzing: {video_file.name}")
    
    # Get video properties using OpenCV
    cap = cv2.VideoCapture(str(video_file))
    
    if cap.isOpened():
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frame_count / fps if fps > 0 else 0
        
        print(f"  📐 Resolution: {width}x{height}")
        print(f"  🎬 FPS: {fps:.2f}")
        print(f"  📽️  Frames: {frame_count}")
        print(f"  ⏱️  Duration: {duration:.2f} seconds")
        
        # Read and display first frame
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(8, 6))
            plt.imshow(frame_rgb)
            plt.title(f"First frame of {video_file.name}")
            plt.axis('off')
            plt.show()
        
        cap.release()
    else:
        print("❌ Could not open video file")
else:
    print("⚠️  No video files available for analysis")

## Video Upscaling with Video2X

### Choose Your Algorithm:
- **Real-ESRGAN**: Best for real-world videos, photographs
- **Anime4K**: Optimized for anime/cartoon content
- **RIFE**: For frame interpolation (increasing FPS)

In [ ]:
# Upscale using Real-ESRGAN (good for real videos)
if input_files:
    input_video = input_files[0]
    output_video = v2x.output_dir / f"{input_video.stem}_realesrgan_2x.mp4"
    
    print(f"🎬 Upscaling {input_video.name} using Real-ESRGAN...")
    print("⚠️  This may take a few minutes depending on video length and complexity")
    
    # Note: This will actually run Video2X via Docker
    # Uncomment the line below to run actual upscaling
    # result = v2x.upscale(str(input_video), str(output_video), processor="realesrgan", scale=2)
    
    # For demo purposes, we'll just show the command that would be run
    print("\n📋 Command that would be executed:")
    print(f"docker run --rm -v {v2x.workspace_dir}:/host ghcr.io/k4yt3x/video2x:latest \\")
    print(f"  -i {input_video} \\")
    print(f"  -o {output_video} \\")
    print(f"  -p realesrgan -s 2 --realesrgan-model realesr-animevideov3")
    
    print("\n💡 To run actual upscaling, uncomment the v2x.upscale() line above")
else:
    print("⚠️  No input videos available")

In [ ]:
# Example for Anime content
if input_files:
    input_video = input_files[0]
    
    print("🎌 For anime content, you would use:")
    print(f"v2x.upscale('{input_video}', processor='anime4k', scale=4)")
    
    print("\n🎬 For frame interpolation (smoother motion):")
    print(f"v2x.interpolate('{input_video}', target_fps=60)")

## Manual Docker Commands

You can also run Video2X directly using Docker commands:

In [ ]:
# Show how to use Video2X directly with Docker
print("🐳 Direct Docker commands for Video2X:")
print()
print("# Real-ESRGAN upscaling (2x):")
print("docker run --rm -v $(pwd):/host ghcr.io/k4yt3x/video2x:latest \\")
print("  -i input/video.mp4 -o output/upscaled.mp4 \\")
print("  -p realesrgan -s 2 --realesrgan-model realesr-animevideov3")
print()
print("# Anime4K upscaling (4x):")
print("docker run --rm -v $(pwd):/host ghcr.io/k4yt3x/video2x:latest \\")
print("  -i input/anime.mp4 -o output/anime_4k.mp4 \\")
print("  -p anime4k -s 4")
print()
print("# Frame interpolation with RIFE:")
print("docker run --rm -v $(pwd):/host ghcr.io/k4yt3x/video2x:latest \\")
print("  -i input/video.mp4 -o output/smooth.mp4 \\")
print("  -p rife")

## Check Output Results

In [ ]:
# List output files
output_files = list(v2x.output_dir.glob('*.mp4')) + list(v2x.output_dir.glob('*.avi')) + list(v2x.output_dir.glob('*.mkv'))

print("📤 Output videos:")
if output_files:
    for i, file in enumerate(output_files):
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  {i+1}. {file.name} ({size_mb:.1f} MB)")
else:
    print("  No output videos yet. Run the upscaling process first.")

print("\n💡 Tips:")
print("- Real-ESRGAN: Best for live-action, realistic content")
print("- Anime4K: Optimized for anime, cartoons, and drawn content")
print("- RIFE: For frame interpolation (increasing framerate)")
print("- Higher scale factors (4x) take much longer than lower ones (2x)")
print("- Process short clips first to test settings before processing long videos")